<a href="https://colab.research.google.com/github/jfarrell8/WQU_5457/blob/main/M3_CodeOutline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Packages

In [2]:
!pip install fredapi

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fredapi
from cvxpy import *

# Data Wrangling

We decided to look at diversifying our portfolio constituents rather than specifically focusing on mining companies. We're going to look at the S&P 500 Dividend Aristocrats. This is an index of the S&P 500 constituents that have increased their dividend yearly for at least the last 25 years. As of today, 03/26/2024, there are 67 companies in this index. Note that this is not a point-in-time representation of the index. Rather, we're taking the current list of 67 constituents, and building our own index with these 67 diversified companies from a past start time.

In [4]:
sp_da = pd.read_html('https://en.wikipedia.org/wiki/S%26P_500_Dividend_Aristocrats')[0]
tickers = sp_da['Ticker symbol'].str.replace('.', '-').tolist()

<ipython-input-4-e6ae386633c7>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tickers = sp_da['Ticker symbol'].str.replace('.', '-').tolist()


In [5]:
sp_da.Sector.unique() # let's look at the different sectors represented in this list...there are 10 --> a decent diversification

array(['Industrials', 'Health Care', 'Financials', 'Materials',
       'Consumer Staples', 'Utilities', 'Information Technology',
       'Energy', 'Real Estate', 'Consumer Discretionary'], dtype=object)

In [6]:
start_date = "2000-01-01"
end_date = "2024-01-14"

data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)['Close']
data.head()

[*********************100%%**********************]  67 of 67 completed


Ticker,ABBV,ABT,ADM,ADP,AFL,ALB,AMCR,AOS,APD,ATO,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,NaN,9.037452,6.413245,24.781534,6.918056,6.173301,NaN,2.339966,17.339005,8.031763,...,4.600554,9.560843,18.910170,15.583126,10.356926,22.670908,9.412900,14.469364,5.428276,18.328705
2000-01-04,NaN,8.779240,6.346441,24.781534,6.734478,6.173301,NaN,2.306249,16.810570,7.831593,...,4.429104,9.435868,18.040506,15.244358,10.034335,21.687786,9.128169,13.927949,5.484354,17.977621
2000-01-05,NaN,8.763098,6.246231,24.543242,6.695827,6.280473,NaN,2.299506,16.909658,8.081805,...,4.443392,9.373381,17.797808,15.041107,10.152211,21.196217,9.111423,13.643712,5.461923,18.957685
2000-01-06,NaN,9.069725,6.279631,24.870874,6.831098,6.494826,NaN,2.265788,18.032562,7.956699,...,4.514826,9.560843,17.312416,14.973356,10.578056,20.193432,9.379401,13.792602,5.405846,19.937767
2000-01-07,NaN,9.166552,6.379840,25.436815,6.966365,6.387652,NaN,2.360196,19.287569,8.031763,...,4.457680,9.623335,17.474213,15.244358,10.595095,21.235542,9.312407,14.834819,5.461923,19.879250


In [7]:
data.tail()

Ticker,ABBV,ABT,ADM,ADP,AFL,ALB,AMCR,AOS,APD,ATO,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-08,159.902237,111.859612,70.394272,235.400208,82.348015,136.145691,9.675838,78.959320,272.839996,118.012146,...,298.802155,127.928101,431.272675,96.960320,75.610001,140.687927,105.802505,52.570953,351.660065,99.990715
2024-01-09,160.773911,112.188026,70.007927,233.869049,80.042633,133.244934,9.636384,79.527092,268.089996,117.525604,...,296.846558,130.099564,429.276825,96.663017,75.580002,142.712936,105.109886,52.923088,349.151398,98.752228
2024-01-10,163.329498,113.849998,68.680481,233.938644,82.119461,132.517273,9.626521,80.164589,266.959991,117.207855,...,297.734558,128.404037,432.230682,97.287354,75.029999,143.030579,106.257652,53.580845,356.967194,97.781258
2024-01-11,162.229996,113.500000,68.819168,233.988358,82.169151,128.240906,9.705426,80.343880,264.589996,114.576538,...,296.756744,126.103668,433.288483,96.018860,75.449997,140.975800,108.107941,53.521053,354.438568,97.761436
2024-01-12,162.399994,113.919998,68.908325,234.594864,82.218834,125.649178,9.616657,80.792114,264.130005,114.973717,...,301.835388,128.404037,436.092651,94.641357,75.750000,139.873962,109.324966,53.590813,346.582794,99.029648


In [8]:
data.shape

(6046, 67)

Let's try to identify start and end dates for each of these constituents to determine if we feel like there's enough data for each

In [9]:
def find_start_end_dates(column):
    start_date = column.first_valid_index()
    end_date = column.last_valid_index()
    return start_date, end_date

In [10]:
price_date_range = pd.DataFrame()
for column in data.columns:
    start_date, end_date = find_start_end_dates(data[column])
    price_date_range = pd.concat([price_date_range, pd.DataFrame([[column, start_date, end_date]], columns=['Ticker', 'StartDate', 'EndDate'])], axis=0)

In [11]:
# any tickers that start really late?
price_date_range.sort_values(by='StartDate', ascending=False)

,Ticker,StartDate,EndDate
0,KVUE,2023-05-04,2024-01-12
0,ABBV,2013-01-02,2024-01-12
0,AMCR,2012-05-15,2024-01-12
0,IBM,2000-01-03,2024-01-12
0,ITW,2000-01-03,2024-01-12
...,...,...,...
0,ESS,2000-01-03,2024-01-12
0,EXPD,2000-01-03,2024-01-12
0,FAST,2000-01-03,2024-01-12
0,FRT,2000-01-03,2024-01-12


In [12]:
# any tickers that get dropped before the current EndDate
price_date_range.EndDate.unique() # NO!

array(['2024-01-12T00:00:00.000000000'], dtype='datetime64[ns]')

In [13]:
# let's drop KVUE, ABBV, AMCR since they start later than 2000-01-03
data = data.drop(['KVUE', 'ABBV', 'AMCR'], axis=1)

In [14]:
data.shape # should be 64 columns now

(6046, 64)

In [15]:
data.head()

Ticker,ABT,ADM,ADP,AFL,ALB,AOS,APD,ATO,BDX,BEN,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,9.037452,6.413245,24.781534,6.918056,6.173301,2.339966,17.339005,8.031763,17.597897,5.742087,...,4.600554,9.560843,18.910170,15.583126,10.356926,22.670908,9.412900,14.469364,5.428276,18.328705
2000-01-04,8.779240,6.346441,24.781534,6.734478,6.173301,2.306249,16.810570,7.831593,17.135891,5.640156,...,4.429104,9.435868,18.040506,15.244358,10.034335,21.687786,9.128169,13.927949,5.484354,17.977621
2000-01-05,8.763098,6.246231,24.543242,6.695827,6.280473,2.299506,16.909658,8.081805,17.261904,5.730762,...,4.443392,9.373381,17.797808,15.041107,10.152211,21.196217,9.111423,13.643712,5.461923,18.957685
2000-01-06,9.069725,6.279631,24.870874,6.831098,6.494826,2.265788,18.032562,7.956699,17.975901,6.081852,...,4.514826,9.560843,17.312416,14.973356,10.578056,20.193432,9.379401,13.792602,5.405846,19.937767
2000-01-07,9.166552,6.379840,25.436815,6.966365,6.387652,2.360196,19.287569,8.031763,18.983881,6.127158,...,4.457680,9.623335,17.474213,15.244358,10.595095,21.235542,9.312407,14.834819,5.461923,19.879250


In [16]:
# let's turn prices into log returns
rets = np.log(data).diff()
rets = rets.dropna()
rets

Ticker,ABT,ADM,ADP,AFL,ALB,AOS,APD,ATO,BDX,BEN,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,-0.028987,-0.010471,0.000000,-0.026895,0.000000,-0.014514,-0.030951,-0.025238,-0.026604,-0.017911,...,-0.037979,-0.013158,-0.047080,-0.021979,-0.031643,-0.044333,-0.030716,-0.038136,0.010278,-0.019341
2000-01-05,-0.001840,-0.015916,-0.009662,-0.005756,0.017212,-0.002928,0.005877,0.031449,0.007327,0.015937,...,0.003221,-0.006644,-0.013544,-0.013423,0.011679,-0.022927,-0.001836,-0.020619,-0.004098,0.053082
2000-01-06,0.034392,0.005333,0.013261,0.020001,0.033561,-0.014772,0.064294,-0.015601,0.040530,0.059461,...,0.015949,0.019802,-0.027651,-0.004515,0.041090,-0.048465,0.028987,0.010854,-0.010320,0.050406
2000-01-07,0.010619,0.015832,0.022500,0.019608,-0.016639,0.040822,0.067282,0.009390,0.054558,0.007422,...,-0.012738,0.006515,0.009302,0.017937,0.001609,0.050319,-0.007168,0.072845,0.010320,-0.002939
2000-01-10,-0.007067,0.000000,0.024293,-0.061491,-0.003361,0.011364,-0.063626,-0.012539,-0.022372,0.005530,...,-0.022692,-0.013072,0.067139,-0.008929,0.003210,-0.019636,0.010734,-0.018417,0.020326,-0.014079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-08,0.014337,0.003524,0.006568,0.001812,0.005506,0.002906,0.009908,0.007347,0.023030,0.015462,...,0.009360,0.007156,0.009859,0.024103,-0.000529,0.006939,0.007133,0.009779,0.035790,-0.016802
2024-01-09,0.002932,-0.005503,-0.006526,-0.028395,-0.021537,0.007165,-0.017563,-0.004131,-0.040741,-0.007186,...,-0.006566,0.016832,-0.004639,-0.003071,-0.000397,0.014291,-0.006568,0.006676,-0.007159,-0.012463
2024-01-10,0.014706,-0.019143,0.000298,0.025616,-0.005476,0.007984,-0.004224,-0.002707,0.002394,-0.001375,...,0.002987,-0.013118,0.006857,0.006438,-0.007304,0.002223,0.010860,0.012352,0.022138,-0.009881


In [17]:
# sanity check
data.ABT/data.ABT.shift(1) - 1 # pretty close to log returns (it shouldn't be exactly equal), so looks good!

Date
2000-01-03         NaN
2000-01-04   -0.028571
2000-01-05   -0.001839
2000-01-06    0.034991
2000-01-07    0.010676
                ...   
2024-01-08    0.014440
2024-01-09    0.002936
2024-01-10    0.014814
2024-01-11   -0.003074
2024-01-12    0.003700
Name: ABT, Length: 6046, dtype: float64

In [ ]:
# add in alternative data/technical indicators
# first need to pivot the rets df


# then add in alternative data


# Data Visualization

In [18]:
# subplots of line plots for each price

In [19]:
# subplots of line plots for each return

In [19]:
# analyze and potentially remove some tickers if need be

# Modeling

## MVO

In [28]:
# get annualized expected returns and covariance matrix
rets_exp = rets.mean()*252
rets_cov = rets.cov()*252

In [29]:
rets_exp

Ticker
ABT     0.105641
ADM     0.098983
ADP     0.093703
AFL     0.103187
ALB     0.125615
          ...   
TGT     0.075857
TROW    0.102228
WMT     0.054583
WST     0.173273
XOM     0.070325
Length: 64, dtype: float64

In [30]:
rets_cov

Ticker,ABT,ADM,ADP,AFL,ALB,AOS,APD,ATO,BDX,BEN,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Ticker,,,,,,,,,,,,,,,,,,,,,
ABT,0.058796,0.021786,0.024986,0.026926,0.027545,0.022013,0.025947,0.017255,0.026082,0.030918,...,0.023202,0.014611,0.027451,0.027524,0.021010,0.021399,0.033415,0.017539,0.024039,0.021432
ADM,0.021786,0.092751,0.028432,0.043386,0.044672,0.032848,0.035901,0.022827,0.021330,0.045998,...,0.027411,0.019099,0.033856,0.039533,0.028046,0.029994,0.046976,0.019806,0.022145,0.037347
ADP,0.024986,0.028432,0.064381,0.040549,0.039778,0.031846,0.034523,0.022416,0.022113,0.043841,...,0.030806,0.017364,0.037180,0.040367,0.030294,0.031131,0.048283,0.021468,0.027294,0.029722
AFL,0.026926,0.043386,0.040549,0.128396,0.055868,0.046045,0.046667,0.029077,0.023560,0.070095,...,0.040401,0.019115,0.049771,0.058722,0.040043,0.041589,0.075217,0.025077,0.031248,0.043385
ALB,0.027545,0.044672,0.039778,0.055868,0.146445,0.053125,0.056061,0.029868,0.027585,0.068760,...,0.044535,0.023316,0.048922,0.062235,0.034520,0.041429,0.070228,0.021903,0.038280,0.044707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGT,0.021399,0.029994,0.031131,0.041589,0.041429,0.037290,0.036368,0.019930,0.018972,0.052388,...,0.035702,0.019583,0.038803,0.045919,0.028499,0.108070,0.057612,0.041959,0.027255,0.026214
TROW,0.033415,0.046976,0.048283,0.075217,0.070228,0.057034,0.055600,0.032085,0.030010,0.098068,...,0.049613,0.025600,0.058908,0.068715,0.040173,0.057612,0.132587,0.033523,0.043314,0.044127
WMT,0.017539,0.019806,0.021468,0.025077,0.021903,0.020886,0.023571,0.015484,0.014913,0.030041,...,0.023030,0.014959,0.023009,0.025045,0.019599,0.041959,0.033523,0.055264,0.017052,0.017959


In [31]:
x = Variable(rets.shape[1])

# return and risk in proper matrix form
ret = rets_exp*x
risk = quad_form(x, rets_cov)

# optimize with constraints (long-only and all weights sum to 1)
prob = Problem(Minimize(risk), [sum(x)==1, x >= 0])

prob.solve()

/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


0.018866501794518844

In [32]:
weights = pd.DataFrame([x.value*100], columns=rets.columns).round(2).T.rename(columns={0:'Pct'})
weights

,Pct
Ticker,
ABT,-0.00
ADM,-0.00
ADP,0.00
AFL,0.00
ALB,0.00
...,...
TGT,0.00
TROW,0.00
WMT,7.04


In [33]:
# show non-zero weights
nz_wts = weights[weights.Pct > 0]
nz_wts = nz_wts.sort_values(by='Pct', ascending=False)
nz_wts

,Pct
Ticker,
ED,15.05
JNJ,12.79
MCD,9.01
HRL,8.07
SJM,7.27
WMT,7.04
BDX,6.86
CHD,4.79
KMB,3.96


In [34]:
nz_wts.sum()

Pct    99.99
dtype: float64

## Neural Nets (MLP, RNN, CNN)

In [34]:
class PortfolioNN:
  def __init__(self):
    # any base args needed
    pass

  def loss_function(self):
    # TBD
    pass

  def train_network(self):
    # train network
    pass

  def rebalance(self):
    # portfolio rebalance
    pass


class MLP(PortfolioNN):
  def __init__(self):
    super.__init__()
    # model architecture here
    pass

class RNN(PortfolioNN):
  def __init__(self):
    super.__init__()
    # model architecture here
    pass

class CNN(PortfolioNN):
  def __init__(self):
    super.__init__()
    # model architecture here
    pass

## Backtest Neural Network Models

## Reinforcement Learning (RL)

Use FinRL vs. Create our Own: TBD

In [42]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git
# !pip install finrl

In [38]:
# from finrl import config
# from finrl import config_tickers
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
import gym

ModuleNotFoundError: No module named 'finrl'

## Build Environment

In [ ]:
class PortolioEnv:
  def __init__(self):
    # TBD what params we need to start with
    pass

  def seed(self):
    # for reproducibility
    pass

  def reset(self):
    # reset the environment
    pass

  def step(self):
    # meat and potatoes of the process
    pass

## Use Different Agents

In [ ]:
# from FinRL tutorial
"""
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)
"""

# this can be done for other agents plus custom agents as well
# custom will be important depending upon how we try new things

## Backtest Models

# Compare Results from RL vs. NNs vs. MVO